In [0]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch import nn
from tqdm import tqdm_notebook as tqdm

In [0]:
# set device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [0]:
class NewsPopularityDataset(Dataset):
  
  def __init__(self, filename):
    self.dataset = pd.read_csv(filename, delimiter=',')
    self.dataset = self.dataset.values
    self.dataset = torch.tensor(self.dataset)
    
    
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    return self.dataset[idx, :-1].float(), self.dataset[idx, -1].long()


In [47]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train_filename = "gdrive/My Drive/Colab Notebooks/OnlineNewsPopularity/train_data.csv"
val_filename = "gdrive/My Drive/Colab Notebooks/OnlineNewsPopularity/test_data.csv"

In [0]:
train_dataset = NewsPopularityDataset(train_filename)
val_dataset = NewsPopularityDataset(val_filename)

In [0]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [51]:
next(iter(train_loader))

[tensor([[3.4638e+04, 8.6000e+01, 7.0000e+00,  ..., 5.0000e-01, 0.0000e+00,
          1.2000e+03],
         [1.8102e+04, 3.7600e+02, 1.1000e+01,  ..., 5.0000e-01, 0.0000e+00,
          2.6000e+03],
         [8.8730e+03, 5.6300e+02, 8.0000e+00,  ..., 4.5455e-02, 1.3636e-01,
          9.3000e+02],
         ...,
         [6.6280e+03, 6.0900e+02, 9.0000e+00,  ..., 5.0000e-01, 0.0000e+00,
          8.2700e+02],
         [2.0140e+03, 6.9600e+02, 1.2000e+01,  ..., 5.0000e-01, 8.5000e-01,
          1.3000e+04],
         [8.1290e+03, 5.7700e+02, 5.0000e+00,  ..., 5.0000e-01, 0.0000e+00,
          2.1000e+03]]),
 tensor([1, 5, 0, 0, 4, 3, 0, 3, 6, 1, 2, 3, 6, 4, 3, 3, 3, 0, 6, 3, 2, 3, 4, 1,
         3, 6, 1, 2, 4, 0, 2, 0, 1, 1, 1, 3, 1, 1, 6, 1, 1, 4, 2, 1, 3, 6, 2, 1,
         0, 0, 0, 2, 5, 1, 0, 6, 3, 1, 6, 5, 0, 3, 0, 0])]

In [0]:
# define model

class Model(nn.Module):
  def __init__(self, num_features, num_classes):
    super(Model, self).__init__()
    
    self.fc1 = nn.Linear(num_features, 64)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(64, 128)
    self.relu2 = nn.ReLU()
    self.fc3 = nn.Linear(128, 256)
    self.relu3 = nn.ReLU()
    self.fc4 = nn.Linear(256, 256)
    self.relu4 = nn.ReLU()
    self.fc5 = nn.Linear(256, 128)
    self.relu5 = nn.ReLU()
    self.fc6 = nn.Linear(128, 64)
    self.relu6 = nn.ReLU()
    self.fc7 = nn.Linear(64, num_classes)
    
  def forward(self, x):
    x = self.relu1(self.fc1(x))
    x = self.relu2(self.fc2(x))
    x = self.relu3(self.fc3(x))
    x = self.relu4(self.fc4(x))
    x = self.relu5(self.fc5(x))
    x = self.relu6(self.fc6(x))
    x = self.fc7(x)
    return x
  
  

In [0]:
# define train method

def train(data_loader, model, optimizer, criterion, curr_epoch, tot_epochs, device):
  
  # set model to training mode
  model.train()
  
  tot_train_loss = 0.0
  
  train_meter = tqdm(
      data_loader, unit=' batches',
      desc=f'[Epoch {curr_epoch}/{tot_epochs}]',
      leave=False
  )
  
  for batch_idx, (sample, label) in enumerate(train_meter):
    
    # transfer to device
    sample, label = sample.to(device), label.to(device)
    
    # zero out gradients
    optimizer.zero_grad()
    
    # forward pass
    out = model(sample)
    
    # compute loss
    loss = criterion(out, label)
    
    # backward
    loss.backward()
    
    # optimizer step
    optimizer.step()
    
    # update tqdm meter
    train_meter.set_postfix(train_loss=f'{loss.item():0.4f}')
    train_meter.update()
    
    # add to total train loss
    tot_train_loss = tot_train_loss + loss.item()
    
  av_train_loss = tot_train_loss / (batch_idx + 1)
  return av_train_loss

In [0]:
# define validation method

def validate(data_loader, model, criterion, device):
  
  # set model to eval mode
  model.eval()
  
  tot_val_loss = 0.0
  corrects = 0
  total = 0
  
  val_meter = tqdm(
      data_loader, unit=' batches',
      desc='val',
      leave=False
  )
  
  with torch.no_grad():
    for batch_idx, (sample, label) in enumerate(val_meter):

      # transfer to device
      sample, label = sample.to(device), label.to(device)

      # forward pass
      out = model(sample)

      # compute loss
      loss = criterion(out, label)
      
      # compute the prediction
      max_value, pred = torch.max(out, dim=-1)
      
      # compute number of correct predictions
      corrects = corrects + (pred == label).sum().item()
      
      # update total samples
      total = total + len(label)

      # update tqdm meter
      val_meter.set_postfix(val_loss=f'{loss.item():0.4f}')
      val_meter.update()

      # add to total val loss
      tot_val_loss = tot_val_loss + loss.item()

  av_val_loss = tot_val_loss / (batch_idx + 1)
  accuracy = corrects / total
  return av_val_loss, accuracy

In [0]:
# specify learning rate
learning_rate = 0.001

# create model object
model = Model(53, 7)

# transfer model to device
model = model.to(device)

# create optimizer
optimizer = torch.optim.SGD(
    filter(lambda param: param.requires_grad, model.parameters()),
    lr=learning_rate
)

# specify criterion
criterion = nn.CrossEntropyLoss()

In [56]:
# specify max number of epochs
num_epochs = 100

train_losses, val_losses, accs = [], [], []

for epoch in range(1, num_epochs + 1):
  train_loss = train(train_loader, model, optimizer, criterion, epoch, num_epochs, device)
  val_loss, accuracy = validate(val_loader, model, criterion, device)
  
  # check for early stopping
  if all(val_loss > loss for loss in val_losses[-3:]) and len(val_losses) > 2: 
    print('stopping early')
    break
  
  # save losses
  train_losses.append(train_loss)
  val_losses.append(val_loss)
  accs.append(accuracy)
  
  # print losses and accuracies
  print(f'[Epoch {epoch}/{num_epochs}] \
             train loss: {train_loss:0.4f}, \
             val loss: {val_loss:0.4f}, \
             accuracy: {accuracy * 100:0.2f} %')

[Epoch 1/100]              train loss: nan,              val loss: nan,              accuracy: 14.2857 %


[Epoch 2/100]              train loss: nan,              val loss: nan,              accuracy: 14.2857 %


[Epoch 3/100]              train loss: nan,              val loss: nan,              accuracy: 14.2857 %


[Epoch 4/100]              train loss: nan,              val loss: nan,              accuracy: 14.2857 %


[Epoch 5/100]              train loss: nan,              val loss: nan,              accuracy: 14.2857 %


...

[Epoch 22/100]              train loss: nan,              val loss: nan,              accuracy: 14.2857 %


[Epoch 23/100]              train loss: nan,              val loss: nan,              accuracy: 14.2857 %


[Epoch 24/100]              train loss: nan,              val loss: nan,              accuracy: 14.2857 %


[Epoch 25/100]              train loss: nan,              val loss: nan,              accuracy: 14.2857 %


KeyboardInterrupt: ignored

In [0]:
# plot losses and accuracies